# Literature

$[1]$ --- Implementable tensor methods in unconstrained convex optimization, https://alfresco.uclouvain.be/alfresco/service/guest/streamDownload/workspace/SpacesStore/aabc2323-0bc1-40d4-9653-1c29971e7bd8/coredp2018_05web.pdf?guest=true.

$[2]$ ---  Cubic regularization of Newton method and its global performance, http://lab7.ipu.ru/files/polyak/Nest_Pol-MathProg'06.pdf.

$[3]$ --- Acelerating the cubic regularization of Newton’s method on convex
problems, http://webdoc.sub.gwdg.de/ebook/serien/e/CORE/dp2005_68.pdf.

$[4]$ --- A.R. Conn, N.I. M. Gould, and Ph.L. Toint. Trust Region Methods. SIAM, Philadelphia, 2000.

$[5]$ --- Анализ быстрого градиентного метода Нестерова для задач машинного обучения с $L_1$-регуляризацией, http://www.machinelearning.ru/wiki/images/0/03/Rodomanov_FGM.pdf.

# Purpose of code: compute method $[1]$.$[2.16]$ with $p = 3$ by testing on logistic regression problem

##### Import libraries

In [1]:
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize import minimize_scalar, minimize
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd.numpy import linalg
from autograd import grad    
from autograd import jacobian
from numpy import linalg
import math
import autograd.scipy as scipy
from scipy import optimize
import copy

##### Calculation parameters $c, A \succeq 0, \gamma > 0$ for auxiliary minimization problem $[1]$.$[5.8]$ in terms of problem 
$$
\langle c, h \rangle + \frac{1}{2}\langle Ah, h \rangle + \frac{\gamma}{4}||h||_2^4 \rightarrow \min\limits_{h \in \mathbb{R}^n}
$$ 

In [5]:
def calc_cur_params(grad_f_cur, hessian_f_cur, tensor3_f_cur_vect, hk, tau, L):
    grad_d = np.dot(hk, hk.T)*hk
    c = (grad_f_cur - 1/tau*np.dot(hessian_f_cur, hk.T) + 0.5*tensor3_f_cur_vect 
                                                                        - tau*(tau + 1)/2*L*grad_d) + tau*tau*L/2*grad_d
    
    A = (tau + 1) / tau * hessian_f_cur
    
    gamma = tau*(tau + 1)/2*L
    return c, A, gamma

##### Objective function $f_{c, A, \gamma}(h) = \langle c, h \rangle + \frac{1}{2}\langle Ah, h \rangle + \frac{\gamma}{4}||h||_2^4$ for auxiliary minimization problem $[1]$.$[5.8]$ and its univariate dual function $g_{c, A, \gamma}(\tau) = \frac{1}{2}\tau^2 + \frac{1}{2}\Bigl\langle \bigl(\sqrt{2\gamma}\tau B + A\bigr)^{-1}c, c \Bigr\rangle$ with $B = I_{n \times n}$ for Euclid 2-norm.

In [6]:
def aux_problem_func(h, *args):
    c, A, gamma = args
    d = linalg.norm(h)
    d4 = d*d*d*d
    return np.dot(c, h) + 0.5 * np.dot(np.dot(A, h), h) + 0.25*gamma*d4 

def aux_problem_onedim_func(tau, *args):
    c, A, B, gamma = args
    sgam = math.sqrt(2*gamma)
    S = sgam*tau*B + A
    invS = linalg.inv(S)
    f = 0.5*tau*tau + 0.5 * np.dot(np.dot(invS, c), c)
    
    return f

##### Calculation minimum of $f_{c, A, \gamma}(h)$ using minimum of dual function $g_{c, A, \gamma}(\tau)$.

In [7]:
def calc_hopt(t, *args):
    c, A, B, gamma = args
    S = math.sqrt(2*gamma)*t*B + A
    invS = linalg.inv(S)
    h_opt = -np.dot(invS, c)
    return h_opt

##### Newton's method for finding minimum $g_{c, A, \gamma}(\tau)$ based on $[4]$, Chapter $7$.

The main idea is to find root $\lambda > 0$ of first-order optimality condition: $\sqrt{\lambda} = ||s(\lambda)||_2, s(\lambda) = -(A + \lambda I)^{-1}c_{\text{new}}, \lambda = \sqrt{2\gamma}\tau, c_{\text{new}} = \sqrt{\gamma}c$.

It is better to solve the secular equation:
$$
\phi(\lambda) = \frac{1}{||s(\lambda)||_{2}} - \frac{1}{\sqrt{\lambda}}.
$$
So 
$$
\phi^{'}(\lambda) = \frac{\bigl \langle s(\lambda), A(\lambda)^{-1}s(\lambda) \bigr \rangle}{||s(\lambda)||_2^3} + \frac{1}{2\lambda^{\frac{3}{2}}} \geq 0, A(\lambda) = A + \lambda I;
$$
$$
\phi^{''}(\lambda) = \frac{3\Bigl(\bigl \langle s(\lambda), \nabla_{\lambda}s(\lambda) \bigr \rangle^2 - ||s(\lambda)||_2^2||\nabla_{\lambda}s(\lambda)||_2^2\Bigr)}{||s(\lambda)||_2^5} - \frac{3}{4\lambda^{\frac{5}{2}}} < 0
$$ by Cauchy-Schwartz inequality.

Newton's method for finding a root of the scalar equation $\phi(\lambda) = 0$ replaces  the estimate $\lambda_k > 0$ with the improved estimate $\lambda_{k + 1}$ for which
$$
\lambda_{k + 1} = \lambda_k - \frac{\phi(\lambda_k)}{\phi^{'}(\lambda_k)}.
$$
Using Cholesky factors $A(\lambda) = L(\lambda)L^T(\lambda)$ we can write:
$$
\bigl \langle s, A(\lambda)^{-1}s \bigr \rangle = \bigl \langle s, L^{-T}L^{-1}s \bigr \rangle = \bigl \langle L^{-1}s, L^{-1}s \bigr \rangle = ||w(\lambda)||_2^2
$$
with $w(\lambda) = L^{-1}(\lambda)s(\lambda)$.
The complete Newton algorithm is:
$$
\lambda_{k + 1} = \lambda_k \cdot \Bigl(1 - 2||s(\lambda_k)||_2^2\frac{\sqrt{\lambda_k} - ||s(\lambda_k)||_2}{||s(\lambda_k)||_2^3 + 2||w(\lambda_k)||_2^2\lambda_k^{\frac{3}{2}}}\Bigr) 
$$ while $\lambda_{k + 1} > 0$.

In [8]:
def Newton_step_tensor3(AuxMaxIter, *args):
    c, H, gamma, eps = args
    c_new = c*(gamma**0.5)
    lambda_ans = eps
    k = 0
    while (k < AuxMaxIter):
        H_lambda = H + lambda_ans*np.identity(H.shape[0])
        try:
            L = linalg.cholesky(H_lambda)
        except np.linalg.LinAlgError:
            break
        s = linalg.solve(H_lambda, -c_new)
        w = linalg.solve(L, s)
        lambda_next = lambda_ans*(1 - 
            2*(lambda_ans**0.5 - linalg.norm(s))*(linalg.norm(s)**2)/
                                  (linalg.norm(s)**3 + 2*(linalg.norm(w)**2)*(lambda_ans**(1.5))))
        if (lambda_next > 0):
            lambda_ans = lambda_next
        else:
            break
        k += 1
    #tau_opt2 = 0
    h_opt1 = -np.dot(linalg.inv(H + lambda_ans*np.identity(H.shape[0])), c)
    #h_opt2 = -np.dot(linalg.pinv(H), c)
    #val_1 = np.dot(c, h_opt1) + 1/2*np.dot(np.dot(H, h_opt1), h_opt1) + gamma/4*(linalg.norm(h_opt1)**4)
    #val_2 = np.dot(c, h_opt2) + 1/2*np.dot(np.dot(H, h_opt2), h_opt2) + gamma/4*(linalg.norm(h_opt2)**4)
    #if (val_1 < val_2):
     #   h_opt = h_opt1
    #else:
      #  h_opt = h_opt2
    #return h_opt
    return h_opt1

##### Iteration of process $[1].[5.8]$ to solve auxiliary minimization problem $[1]$.$[5.4]$.

In [9]:
def T3M(x, aux_prob_method, grad_f_cur, hess_f_cur, tensor3_f_direct, TensorMaxIter, AuxMaxIter, tau, L, eps, *args):
    
    n = x.shape[0]
    hk = np.zeros(n)
    B = np.eye(n)
 
    if aux_prob_method == 1:
        xb = L*1e10
        t0 = 0+eps
    
    for aux_k in range(1, TensorMaxIter + 1):
        tensor3_f_cur_vect = tensor3_f_direct(x, hk, *args)
        c, A, gamma = calc_cur_params(grad_f_cur, hess_f_cur, tensor3_f_cur_vect, hk, tau, L)
            
        if aux_prob_method == 1:
            allargs = (c, A, B, gamma)

            res = optimize.minimize_scalar(aux_problem_onedim_func, args = allargs, method = 'bounded', bounds=(t0, xb), 
                                                options={'xatol': eps, 'maxiter': AuxMaxIter, 'disp': False})
            
            tau_opt = res.x
            
            h_opt = calc_hopt(tau_opt, *(c, A, B, gamma))
                
        elif aux_prob_method == 2:
            allargs = (c, A, gamma)
        
            res = optimize.minimize(aux_problem_func, args = allargs, tol = eps, x0 = hk, method = 'Powell', 
                                                options={'ftol': eps,'maxiter': AuxMaxIter, 'disp': False})                
            h_opt = res.x
                
        elif aux_prob_method == 3:
            h_opt = Newton_step_tensor3(AuxMaxIter, *(c, A, gamma, eps))
        hk = h_opt
    return (hk + x)      

##### Tensor method $[1]$.$[2.16]$ by Yu. Nesterov (2018), $p = 3$
Parameters:  
$NumIter$ --- max number of steps in tensor method.

$TensorMaxIter$ --- max number of iterations in auxiliary problem solution method $[1]$.$[5.8]$.

$AuxMaxIter$ --- max number of iterations to perform step $[1]$.$[5.8]$.

$x_0$ --- initial point.

$f$ --- objective function oracle.  

$\nabla f$ --- objective gradient of $f$ oracle.

$\nabla^2 f$ --- objective hessian of $f$ oracle.

$\langle \nabla^3 f(x) h, h \rangle$ --- objective third directional derivative of $f$ at $x$ along direction $h$ oracle .

$\tau > 1$ - parameter in $[1]$.$[5.5]$.

$L_3(f)$ - uniform bound for the Lipschitz constant of third derivative.

$\varepsilon$ - parameter to perform step $[1]$.$[5.8]$, meaning depends of $aux\_prob\_method$.

$aux\_prob\_method$ --- how to solve auxiliary problem (5.4): $1$ --- by means of minimization $g_{c, A, \gamma}(\tau)$ using $scipy.optimize$, $2$ --- by means of minimization $f_{c, A, \gamma}(h)$ using $scipy.optimize$, $3$ --- by means of minimization $g_{c, A, \gamma}(\tau)$ using Newton's method.

$args$ - arguments for parametrization of $f$.

In [10]:
def Tensor3_iter(NumIter, TensorMaxIter, AuxMaxIter, x0, f, grad_f, hess_f, tensor3_f_direct, tau, L, eps, 
                                                             aux_prob_method, *args):
    
    farr = np.zeros(NumIter + 1)
    #fgradarr = np.zeros(NumIter + 1)
    xk = copy.deepcopy(x0)

    for k in range(NumIter):
        f_xk = f(xk, *args)
        grad_f_xk = grad_f(xk, *args)
        hessian_f_xk = hess_f(xk, *args)
        farr[k] = f_xk
        #fgradarr[k] = linalg.norm(grad_f_xk)
        xk = T3M(xk, aux_prob_method, grad_f_xk, hessian_f_xk, tensor3_f_direct, TensorMaxIter, AuxMaxIter,
                                                             tau, L, eps, *args) 
    
    f_xk = f(xk, *args)
    #grad_f_xk = grad_f(xk, *args)
    farr[NumIter] = f_xk
    #fgradarr[NumIter] = linalg.norm(grad_f_xk)
    
    return xk, f_xk, farr#, fgradarr
    
    

##### Tensor method $[1]$.$[2.16]$ by Yu. Nesterov (2018), $p = 3$ with history printing

In [3]:
def Tensor3_iter_hist(NumIter, TensorMaxIter, AuxMaxIter, x0, f, grad_f, hess_f, tensor3_f_direct, tau, L, eps, 
                                                             aux_prob_method, *args):
    
    farr = np.zeros(NumIter + 1)
    #fgradarr = np.zeros(NumIter + 1)
    xk = copy.deepcopy(x0)

    for k in range(NumIter):
        f_xk = f(xk, *args)
        print("iter = {0}, f_k = {1}".format(k, f_xk))
        grad_f_xk = grad_f(xk, *args)
        hessian_f_xk = hess_f(xk, *args)
        farr[k] = f_xk
        #fgradarr[k] = linalg.norm(grad_f_xk)
        xk = T3M(xk, aux_prob_method, grad_f_xk, hessian_f_xk, tensor3_f_direct, TensorMaxIter, AuxMaxIter,
                                                             tau, L, eps, *args) 
    
    f_xk = f(xk, *args)
    print("iter = {0}, f_k = {1}".format(k, f_xk))
    #grad_f_xk = grad_f(xk, *args)
    farr[NumIter] = f_xk
    #fgradarr[NumIter] = linalg.norm(grad_f_xk)
    
    return xk, f_xk, farr#, fgradarr

def TensorAcc3_iter_hist(NumIter, TensorMaxIter, AuxMaxIter, x0, f, grad_f, hess_f, tensor3_f_direct, tau, L, eps, 
                                               aux_prob_method, version, *args):
    
    xk = copy.deepcopy(x0)
    M = tau*tau*L
    
    farr = np.zeros(NumIter + 1)
    #fgradarr = np.zeros(NumIter + 1)
    
    
    f_xk = f(xk, *args)
    print("iter = {0}, f_k = {1}".format(0, f_xk))
    farr[0] = f_xk
    grad_f_xk = grad_f(xk, *args)
    #fgradarr[0] = linalg.norm(grad_f_xk)
    hessian_f_xk = hess_f(xk, *args)
  
    xk = T3M(xk, aux_prob_method, grad_f_xk, hessian_f_xk, tensor3_f_direct, TensorMaxIter, AuxMaxIter, tau, L, 
                                                                                                         eps, *args)  
    #print('xk = {0}'.format(xk))
    f_xk = f(xk, *args)
    print("iter = {0}, f_k = {1}".format(1, f_xk))
    #gradf_xk = grad_f(xk, *args)
    farr[1] = f_xk
    #fgradarr[1] = linalg.norm(gradf_xk)
    
    p = 3
    C = p / 2 * math.sqrt((p+1)/(p-1)*(M*M - L*L))
    
    min_psik = copy.deepcopy(x0)
    size = x0.shape[0]
    sk = np.zeros(size)
    if (version == 1):
        ak_part = math.sqrt(pow((p - 1) * (M*M - L*L) / 4 / (p + 1) / M / M, p))
    elif (version == 2):
        ak_part = math.sqrt(pow((p + 1) * (M*M - L*L) / 4 / (p - 1) / M / M, p))
    k = 1
    Ak1 = ak_part*pow(k / (p + 1), p + 1)
    
    factor_p = math.factorial(p)
    
    for k in range(1, NumIter):
        vk = min_psik
        Ak = Ak1
        Ak1 = ak_part*pow((k + 1) / (p + 1), p + 1)
        alpha = Ak / Ak1
        #print('alpha = {0}'.format(alpha))
        yk = alpha * xk + (1 - alpha) * vk
        #print('yk = {0}'.format(yk))
        grad_f_yk = grad_f(yk, *args)
        hessian_f_yk = hess_f(yk, *args)

        xk = T3M(yk, aux_prob_method, grad_f_yk, hessian_f_yk, tensor3_f_direct, TensorMaxIter, AuxMaxIter, 
                                                                                     tau, L, eps, *args)        
        #print('xk = {0}'.format(xk))
        grad_next = grad_f(xk, *args)
        a = Ak1 - Ak
        sk = sk + a * grad_next 
        min_psik = x0 - pow(factor_p / C / pow(linalg.norm(sk), p - 1), 1. / p) * sk
        
        f_xk = f(xk, *args)
        print("iter = {0}, f_k = {1}".format(k + 1, f_xk))
        farr[k + 1] = f_xk
        #fgradarr[k + 1] = linalg.norm(grad_next)
    
    return xk, f_xk, farr#, fgradarr, k

# Logistic regression

## Reading data

In [65]:
features = 54
train_object_size = 20000
test_object_size = 30000
read_object_size = train_object_size + test_object_size

train_object = np.zeros((train_object_size, features))
train_ans = np.zeros(train_object_size)

test_object = np.zeros((test_object_size, features)) 
test_ans = np.zeros(test_object_size)

f = open('covtype.libsvm.binary.scale')
line_num = 0
for line in f:
    if (line_num == read_object_size):
        break
    line_object = line.split()
    len_line_object = len(line_object)
    for i in range(1, len_line_object):
        current_cell = line_object[i].split(':')
        current_num_feature = int(current_cell[0]) - 1
        current_feature = float(current_cell[1])
        bin_class = int(line_object[0])
        if (bin_class == 2):
            bin_class = -1
        if (line_num < train_object_size):
            train_ans[line_num] = bin_class
            train_object[line_num][current_num_feature] = current_feature
        else:
            test_ans[line_num - train_object_size] = bin_class
            test_object[line_num - train_object_size][current_num_feature] = current_feature
    line_num += 1
f.close()
print("Data was read successfully!")

Data was read successfully!


##### Logististic loss and it's gradient, hessian and third derivative using closed-form expression

In [66]:
def logistic_loss(w, *args):
    X, y = args
    objects_size = y.shape[0]
    return sum([np.log(1 + np.exp(-y[i]*np.dot(X[i], w))) for i in range(objects_size)])

def grad_logistic_loss(w, *args):
    X, y = args
    objects_size = y.shape[0]
    return sum([-y[i]*X[i]/(1 + np.exp(y[i]*np.dot(X[i], w))) for i in range(objects_size)])

def hess_logistic_loss(w, *args):
    X, y = args
    objects_size = y.shape[0]
    features = X.shape[1]
    ans = np.zeros((features, features))
    for i in range(objects_size):
        ans += np.exp(y[i]*np.dot(X[i], w))/((1 + np.exp(y[i]*np.dot(X[i], w)))**2)*np.dot(X[i].reshape(-1, 1),
                                                                X[i].reshape(1, -1))
    return ans

def dot_tensor3_and_vector_vector_logistic_loss(w, h, *args):
    X, y = args
    objects_size = y.shape[0]
    features = X.shape[1]
    ans = np.zeros(features)
    for i in range(objects_size):
        add = y[i]*(1 - np.exp(y[i]*np.dot(X[i], w)))*(np.dot(h, X[i])**2)*np.exp(y[i]*np.dot(X[i], w))
        add = add/((1 + np.exp(y[i]*np.dot(X[i], w)))**3)*X[i]
        ans += add
    return ans


##### Upper bounds on $L_2(\text{logistic loss})$ and $L_3(\text{logistic loss})$

In [67]:
def L2_upper_bound(X):
    return 1/10*sum([linalg.norm(X[i])**3 for i in range(X.shape[0])])

def L3_upper_bound(X):
    return 1/8*sum([linalg.norm(X[i])**4 for i in range(X.shape[0])])

# Our problem to solve and it's parametres

In [68]:
train_const_feature = np.array([np.array([1]) for i in range(train_object_size)]) #добавим фиктивный признак
train_extended_object = copy.deepcopy(train_object)
train_extended_object = np.column_stack((train_extended_object, train_const_feature[:, 0]))
args = (train_extended_object, train_ans)

features_num = features + 1
w0 = np.zeros(features_num)

NumIter = 100
TensorMaxIter = 40
AuxMaxIter = 100
eps = 1e-7
tau = 1 + eps
L2 = L2_upper_bound(train_extended_object)
L3 = L3_upper_bound(train_extended_object)

In [69]:
L2

28193.786578212414

In [70]:
L3

85588.5309823477

In [71]:
train_extended_object[0]

array([ 0.368684 ,  0.141667 ,  0.0454545,  0.184681 ,  0.223514 ,
        0.0716594,  0.870079 ,  0.913386 ,  0.582677 ,  0.875366 ,
        1.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  1.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  1.       ])

# Experiments with different methods

In [78]:
aux_prob_method = 1
xans_tensor3, farr_tensor3 = Tensor3_iter_hist(NumIter, TensorMaxIter, AuxMaxIter, w0, logistic_loss, grad_logistic_loss,
            hess_logistic_loss, dot_tensor3_and_vector_vector_logistic_loss, tau, L3, eps, aux_prob_method, *args)


iter = 0, f_k = 13862.943611201723
iter = 1, f_k = 11421.663356650139
iter = 2, f_k = 10254.520099817568
iter = 3, f_k = 9516.404499219063
iter = 4, f_k = 9024.1916097374
iter = 5, f_k = 8691.008552110596
iter = 6, f_k = 8456.964221117521
iter = 7, f_k = 8282.48930277633
iter = 8, f_k = 8143.950843584272
iter = 9, f_k = 8028.366429968638
iter = 10, f_k = 7928.833863479684
iter = 11, f_k = 7841.554509547678
iter = 12, f_k = 7764.232460384513
iter = 13, f_k = 7695.305884654516
iter = 14, f_k = 7633.598227076089
iter = 15, f_k = 7578.161294012672
iter = 16, f_k = 7528.201355157959
iter = 17, f_k = 7483.04053941726
iter = 18, f_k = 7442.093713976898
iter = 19, f_k = 7404.852816939839
iter = 20, f_k = 7370.875308689801
iter = 21, f_k = 7339.775186805825
iter = 22, f_k = 7311.215725052544
iter = 23, f_k = 7284.903416433555
iter = 24, f_k = 7260.58276691386
iter = 25, f_k = 7238.031722386116
iter = 26, f_k = 7217.057577336579
iter = 27, f_k = 7197.493282026668
iter = 28, f_k = 7179.1941000414

##### Saving data

In [79]:
farr_tensor3

array([ 13862.9436112 ,  11421.66335665,  10254.52009982,   9516.40449922,
         9024.19160974,   8691.00855211,   8456.96422112,   8282.48930278,
         8143.95084358,   8028.36642997,   7928.83386348,   7841.55450955,
         7764.23246038,   7695.30588465,   7633.59822708,   7578.16129401,
         7528.20135516,   7483.04053942,   7442.09371398,   7404.85281694,
         7370.87530869,   7339.77518681,   7311.21572505,   7284.90341643,
         7260.58276691,   7238.03172239,   7217.05757734,   7197.49328203,
         7179.19410004,   7162.03459547,   7145.90592992,   7130.71346653,
         7116.37466613,   7102.81725879,   7089.97767331,   7077.79970141,
         7066.23337057,   7055.23400208,   7044.76142714,   7034.77933965,
         7025.25476235,   7016.15760754,   7007.46031645,   6999.13756178,
         6991.1660017 ,   6983.52407512,   6976.1918299 ,   6969.15077688,
         6962.38376593,   6955.87487652,   6949.60932231,   6943.57336622,
         6937.75424344,  

In [80]:
np.savetxt('tensor3_result_log_regr.txt', farr_tensor3)

# Comparing autograd and formulas

In [52]:
grad_logistic_loss_auto = grad(logistic_loss)
hess_logistic_loss_auto = jacobian(grad_logistic_loss_auto)
tensor3_logistic_loss_auto = jacobian(hess_logistic_loss_auto)

print(grad_logistic_loss(w0, *args))
print(grad_logistic_loss_auto(w0, *args))

[ -1.56662864e+03  -1.86405556e+03  -1.15640142e+03  -6.42997164e+02
  -1.22648108e+03  -4.01957061e+02  -3.28178577e+03  -3.35611671e+03
  -2.06313579e+03   8.35405535e+01   7.57500000e+02  -1.83500000e+02
  -2.23450000e+03  -2.31750000e+03  -1.77500000e+02  -3.08500000e+02
  -4.69000000e+02  -4.01500000e+02  -8.25000000e+01  -3.18000000e+02
   0.00000000e+00   5.00000000e-01   4.00000000e+00  -9.90000000e+02
  -1.36000000e+02   2.72000000e+02  -1.54000000e+02  -8.45000000e+01
   0.00000000e+00   7.75000000e+01  -2.99000000e+02   2.90000000e+01
  -1.00000000e+01  -1.00000000e+01  -8.00000000e+00  -1.18500000e+02
  -2.29500000e+02  -5.20000000e+01   5.00000000e-01  -8.00000000e+00
  -2.50000000e+00   1.50000000e+00   6.05500000e+02   1.51500000e+02
  -6.90000000e+01  -9.00000000e+01  -1.24000000e+02   1.00000000e+00
  -5.00000000e+01  -3.00000000e+00  -1.70000000e+01  -3.57000000e+02
  -3.25500000e+02  -2.26500000e+02  -3.97800000e+03]
[ -1.56662864e+03  -1.86405556e+03  -1.15640142e+0

In [56]:
print(hess_logistic_loss(w0, *args)[0])
print(hess_logistic_loss_auto(w0, *args)[0])

[  1.16029057e+03   9.29797380e+02   4.99013647e+02   3.77857408e+02
   6.32462807e+02   7.37414754e+02   1.87871855e+03   1.93684404e+03
   1.19865850e+03   8.24095873e+02   1.05332353e+03   9.24887368e+01
   8.44931576e+02   2.34505449e+02   1.33599355e+01   4.61442049e+01
   4.78331817e+01   7.11512087e+01   6.76350986e+00   4.23619403e+01
   0.00000000e+00   1.30190000e-01   8.98824000e-01   1.39959510e+02
   4.03670595e+01   1.32846554e+02   5.84209612e+01   7.14194745e+00
   0.00000000e+00   4.11544535e+01   3.69115846e+01   3.07417492e+01
   6.41845875e+00   2.81361940e+01   2.61630775e+00   5.56600773e+01
   1.11824784e+02   3.97098530e+01   1.79339750e-01   6.98649375e+00
   2.66520775e+00   9.81366000e-01   4.43180969e+02   1.89776510e+02
   4.77526253e+01   1.05235488e+02   8.79214560e+01   3.10817850e+00
   1.91253100e+01   1.93721875e+00   6.54752300e+00   1.35745613e+02
   1.20880056e+02   9.26734525e+01   2.22524930e+03]
[  1.16029057e+03   9.29797380e+02   4.99013647e+0

In [101]:
train_extended_object = np.array([[1.2, 2.3, 3.4, 2.8], [19.7, 5.5, 6.6, 7.7]])
train_ans = np.array([1, -1])
w0 = np.array([5.6, 1.3, 5, 1.2])
h = np.array([1.1, 1.9, 12.4, 1.7])
args = (train_extended_object, train_ans)
tensor_3_auto = tensor3_logistic_loss_auto(w0, *args)
print(np.dot(np.dot(tensor_3_auto, h), h))
print(dot_tensor3_and_vector_vector_logistic_loss(w0, h, *args))

[ -2.75127447e-10  -5.76882441e-10  -7.95231482e-10  -6.87292495e-10]
[ -2.89789589e-10  -5.55430045e-10  -8.21070502e-10  -6.76175707e-10]


In [103]:
train_const_feature = np.array([np.array([1]) for i in range(train_object_size)]) #добавим фиктивный признак
train_extended_object = copy.deepcopy(train_object)
train_extended_object = np.column_stack((train_extended_object, train_const_feature[:, 0]))
args = (train_extended_object, train_ans)

features_num = features + 1
w0 = np.ones(features_num)
h = np.ones(features_num)
tensor_3_auto = tensor3_logistic_loss_auto(w0, *args)
print(np.dot(np.dot(tensor_3_auto, h), h))
print(dot_tensor3_and_vector_vector_logistic_loss(w0, h, *args))

[-0.02729094 -0.01105561 -0.00280546 -0.01248178 -0.01632065 -0.00468862
 -0.06452449 -0.06833928 -0.0437575  -0.06478162 -0.07433052  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.         -0.07433052  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.07433052]
[-0.02729094 -0.01105561 -0.00280546 -0.01248178 -0.01632065 -0.00468862
 -0.06452449 -0.06833928 -0.0437575  -0.06478162 -0.07433052  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0. 